In [3]:
%tensorflow_version 1.x
%cd /content
!git clone https://github.com/tomyaacov/RPNIComposition
%cd RPNIComposition

TensorFlow is already loaded. Please restart the runtime to change versions.
/content
fatal: destination path 'RPNIComposition' already exists and is not an empty directory.
/content/RPNIComposition


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense, Dropout, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import time
import sys
import random
random.seed(1)

In [5]:
data = []
with open("data/" + "accepted_words.csv", 'r') as f:
    for line in f:
        data.append(line.strip().split(",") + ["1"])
with open("data/" + "rejected_words.csv", 'r') as f:
    for line in f:
        data.append(line.strip().split(",") + ["0"])
random.shuffle(data)
train = data[:int(len(data) * 0.8)]
test = data[int(len(data) * 0.8):]

In [6]:
train_data = [x[:-1] for x in train]
test_data = [x[:-1] for x in test]
train_labels = np.array([int(x[-1]) for x in train])
test_labels = np.array([int(x[-1]) for x in test])

In [7]:
def get_enc(c):
  if c[0] == "X":
    return int(c[1:])
  else:
    return 9+int(c[1:])
train_data_enc = [[get_enc(c) for c in d] for d in train_data]
test_data_enc = [[get_enc(c) for c in d] for d in test_data]


In [8]:
max_length = 9
train_data_pad = pad_sequences(train_data_enc,maxlen=max_length,padding='post',value=18)
test_data_pad = pad_sequences(test_data_enc,maxlen=max_length,padding='post',value=18)

In [9]:
model=Sequential()

model.add(Embedding(19,10,input_length=9))
model.add(LSTM(32,input_shape=(9, 10),activation='relu',return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(32,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 9, 10)             190       
                                                                 
 lstm (LSTM)                 (None, 9, 32)             5504      
                                                                 
 dropout (Dropout)           (None, 9, 32)             0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 32)                1056      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0

In [13]:
start_time = time.time()
history = model.fit(train_data_pad, train_labels, epochs=10, batch_size=64, validation_data=(test_data_pad, test_labels))
print("lstm:")
print("--- %s seconds training ---" % (time.time() - start_time))
print("lstm acc", history.history["accuracy"])
lstm_acc = history.history["accuracy"]
print("lstm val acc", history.history["val_accuracy"])
lstm_val_acc = history.history["val_accuracy"]

Epoch 1/10
23652/23652 [==============================] - 917s 39ms/step - loss: 0.0202 - accuracy: 0.9942 - val_loss: 0.0166 - val_accuracy: 0.9952
Epoch 2/10
23652/23652 [==============================] - 915s 39ms/step - loss: 0.0153 - accuracy: 0.9956 - val_loss: 0.0132 - val_accuracy: 0.9960
Epoch 3/10
 8969/23652 [==========>...................] - ETA: 9:07 - loss: 0.0137 - accuracy: 0.9959

KeyboardInterrupt: ignored

In [22]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [24]:
import pandas as pd
path = '/content/drive/MyDrive/research/tic_tac_toe_model_history.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  df = pd.DataFrame.from_dict(history.history)
  df.to_csv(f)

path = '/content/drive/MyDrive/research/tic_tac_toe_model.h5'
model.save(path)